In [2]:
# -*- coding: utf-8 -*-
import os, re, datetime
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

# =========================
# GPU セットアップ（4090想定）
# =========================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"[INFO] GPUs found: {len(gpus)} -> memory growth enabled")
    except Exception as e:
        print("[WARN] set_memory_growth failed:", e)
else:
    print("[WARN] No GPU detected. Running on CPU.")

# =========================
# パス設定（必要に応じて変更）
# =========================
PROCESS_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\process"

# 使うサブフォルダ（例：正規化図）
SRC_SUBDIRS = [
    "left_paw_nor",
    "right_paw_nor",
    # "left_tarsal_nor",
    # "right_tarsal_nor",
    # 生の軌跡を混ぜたいなら "left_paw" なども可
]

# 出力先
MODEL_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\models_cnn_traj"
CURVE_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\output\learning_curve_traj"
EVAL_DIR  = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\output\learning_curve_traj"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CURVE_DIR, exist_ok=True)
os.makedirs(EVAL_DIR,  exist_ok=True)

# =========================
# ハイパパラメータ
# =========================
IMG_SIZE = (300, 300)
BATCH_SIZE = 16
EPOCHS = 200
VAL_RATIO = 0.3
LEARNING_RATE = 1e-6
AUGMENT = True

# =========================
# 3クラス定義（順序は要求に合わせて one, two, normal）
# =========================
CLASSES = ["one", "two", "normal"]
NAME2IDX = {c:i for i,c in enumerate(CLASSES)}

def infer_label_from_path(p: Path):
    """
    one / two / normal の 3クラスを推定。
    互換のため以下も受け付ける:
      - 'ivdd1' => one, 'ivdd2' => two
      - 'grade1','g1' => one, 'grade2','g2' => two
    ファイル名 or 親ディレクトリ名のトークンから判定（厳密一致）。
    """
    def tokens(s):
        return [t for t in re.split(r"[^a-z0-9]+", s.lower()) if t]

    toks = set(tokens(p.stem)) | set(tokens(p.name)) | set(tokens(p.parent.name))

    one_keys = {"one", "ivdd1", "g1", "grade1"}
    two_keys = {"two", "ivdd2", "g2", "grade2"}
    normal_keys = {"normal"}

    hit_one = len(one_keys & toks) > 0
    hit_two = len(two_keys & toks) > 0
    hit_norm = len(normal_keys & toks) > 0

    # 優先順位: one / two / normal（重複は基本起こらない想定）
    if hit_one and not hit_two and not hit_norm:
        return NAME2IDX["one"]
    if hit_two and not hit_one and not hit_norm:
        return NAME2IDX["two"]
    if hit_norm and not hit_one and not hit_two:
        return NAME2IDX["normal"]

    # あいまい・未検出は None
    return None

# =========================
# 画像リスト収集
# =========================
def collect_images(process_dir: str, subdirs=None, exts=(".png", ".jpg", ".jpeg")):
    proc = Path(process_dir)
    paths, labels = [], []
    subdirs = list(subdirs) if subdirs else []

    if subdirs:
        cand_dirs = [proc / sd for sd in subdirs]
    else:
        cand_dirs = [d for d in proc.iterdir() if d.is_dir()]

    for d in cand_dirs:
        if not d.exists():
            print(f"[WARN] ディレクトリが見つかりません: {d}")
            continue
        for p in d.rglob("*"):
            if p.is_file() and p.suffix.lower() in exts:
                y = infer_label_from_path(p)
                if y is None:
                    print(f"[SKIP] ラベル不明: {p}")
                    continue
                paths.append(str(p))
                labels.append(y)

    if not paths:
        raise FileNotFoundError("画像が見つかりませんでした。ファイル名/親フォルダ名に one/two/normal（または ivdd1/ivdd2）を含めてください。")

    df = pd.DataFrame({"path": paths, "y": labels})
    return df

df_all = collect_images(PROCESS_DIR, SRC_SUBDIRS)
print("[INFO] 収集画像数:", len(df_all))
print(df_all["y"].value_counts().rename(index={i:c for c,i in NAME2IDX.items()}))

# =========================
# Stratified Train/Val 分割
# =========================
sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_RATIO, random_state=42)
train_idx, val_idx = next(sss.split(df_all["path"], df_all["y"]))
df_tr = df_all.iloc[train_idx].reset_index(drop=True)
df_va = df_all.iloc[val_idx].reset_index(drop=True)
print(f"[INFO] split -> train: {len(df_tr)}, val: {len(df_va)}")

# =========================
# tf.data パイプライン
# =========================
AUTOTUNE = tf.data.AUTOTUNE

def decode_img(path):
    img_bytes = tf.io.read_file(path)
    img = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.clip_by_value(img/255.0, 0.0, 1.0)
    return img

def aug(img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.05)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
    return img

def make_ds(paths, labels, training=True, batch_size=BATCH_SIZE):
    ds_paths = tf.data.Dataset.from_tensor_slices((paths, labels))
    def _load(path, y):
        img = decode_img(path)
        if training and AUGMENT:
            img = aug(img)
        y_onehot = tf.one_hot(y, depth=len(CLASSES))
        return img, y_onehot
    ds = ds_paths.map(_load, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(buffer_size=min(1000, len(paths)))
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(df_tr["path"].values, df_tr["y"].values, training=True)
val_ds   = make_ds(df_va["path"].values, df_va["y"].values, training=False)

# =========================
# class_weight（不均衡対策）
# =========================
unique_tr = np.unique(df_tr["y"].values)
if set(unique_tr) == {0,1,2}:
    cls_w = compute_class_weight(
        class_weight="balanced",
        classes=np.array([0,1,2]),
        y=df_tr["y"].values
    )
    class_weight = {0: float(cls_w[0]), 1: float(cls_w[1]), 2: float(cls_w[2])}
else:
    print("[WARN] 学習セットに全クラスが揃っていません。class_weight を無効化します。")
    class_weight = None
print("[INFO] class_weight:", class_weight)

# =========================
# CNN モデル（軽く強化：BatchNorm 追加）
# =========================
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # ← 3クラス（one, two, normal）
])

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# =========================
# 学習
# =========================
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = os.path.join(MODEL_DIR, f"traj_cnn_3cls_{now}.h5")
curve_path = os.path.join(CURVE_DIR, f"traj_learning_curve_3cls_{now}.png")
pred_csv   = os.path.join(EVAL_DIR,  f"traj_val_predictions_3cls_{now}.csv")
cm_png     = os.path.join(EVAL_DIR,  f"traj_cm_3cls_{now}.png")

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min", factor=0.5, patience=5, min_lr=1e-6, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        model_path, monitor="val_accuracy", mode="max", save_best_only=True, verbose=1
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight,
    verbose=1,
    callbacks=callbacks
)

print(f"✅ モデル保存: {model_path}")

# =========================
# 学習曲線保存
# =========================
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Loss"); plt.legend(); plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy"); plt.legend(); plt.grid(True)

plt.tight_layout()
plt.savefig(curve_path, dpi=150)
plt.close()
print(f"✅ 学習曲線保存: {curve_path}")

# =========================
# 検証データでの詳細評価
# =========================
val_paths  = df_va["path"].values
val_labels = df_va["y"].values
y_true, y_prob, y_pred = [], [], []

for start in range(0, len(val_paths), BATCH_SIZE):
    batch_paths = val_paths[start:start+BATCH_SIZE]
    batch_imgs = np.stack([
        tf.image.resize(
            tf.io.decode_image(tf.io.read_file(p), channels=3, expand_animations=False),
            IMG_SIZE
        ).numpy()/255.0
        for p in batch_paths
    ], axis=0)
    probs = model.predict(batch_imgs, verbose=0)  # (b,3)
    preds = probs.argmax(axis=1)
    y_prob.append(probs)
    y_pred.append(preds)
    y_true.append(val_labels[start:start+BATCH_SIZE])

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
y_prob = np.concatenate(y_prob)

print("\n[Validation] classification_report")
print(classification_report(y_true, y_pred, target_names=CLASSES, digits=4))

# 予測CSV
df_pred = pd.DataFrame({
    "path": val_paths,
    "true": [CLASSES[i] for i in y_true],
    "pred": [CLASSES[i] for i in y_pred],
    "p_one":    y_prob[:, NAME2IDX["one"]],
    "p_two":    y_prob[:, NAME2IDX["two"]],
    "p_normal": y_prob[:, NAME2IDX["normal"]],
})
df_pred.to_csv(pred_csv, index=False, encoding="utf-8-sig")
print(f"✅ 予測CSV保存: {pred_csv}")

# 混同行列PNG（生カウントを表示）
cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
fig, ax = plt.subplots(figsize=(7,6))
im = ax.imshow(cm.astype(np.float32) / max(cm.sum(),1), cmap="Blues")
ax.figure.colorbar(im, ax=ax)
ax.set_xticks([0,1,2]); ax.set_yticks([0,1,2])
ax.set_xticklabels(CLASSES); ax.set_yticklabels(CLASSES)
ax.set_xlabel("Predicted"); ax.set_ylabel("True")
ax.set_title("Confusion Matrix (Validation)")
for i in range(3):
    for j in range(3):
        ax.text(j, i, int(cm[i,j]),
                ha="center",
                color="white" if cm[i,j] > cm.max()/2 else "black",
                fontsize=13)
plt.tight_layout()
plt.savefig(cm_png, dpi=150); plt.close()
print(f"✅ 混同行列保存: {cm_png}")


[WARN] No GPU detected. Running on CPU.
[INFO] 収集画像数: 2422
y
normal    1494
one        526
two        402
Name: count, dtype: int64
[INFO] split -> train: 1695, val: 727
[INFO] class_weight: {0: 1.5353260869565217, 1: 2.0106761565836297, 2: 0.5401529636711281}


c:\kanno\envs\MLya\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 298, 298, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 298, 298, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 147, 147, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 71, 71, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    20,070,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,165,059 (76.92 MB)

...

106/106 ━━━━━━━━━━━━━━━━━━━━ 55s 507ms/step - accuracy: 0.9952 - loss: 0.0276 - val_accuracy: 0.9615 - val_loss: 0.1105 - learning_rate: 1.0000e-06
Epoch 117/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9963 - loss: 0.0242
Epoch 117: val_accuracy did not improve from 0.96149
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 503ms/step - accuracy: 0.9963 - loss: 0.0242 - val_accuracy: 0.9601 - val_loss: 0.1109 - learning_rate: 1.0000e-06
Epoch 118/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.9994 - loss: 0.0184
Epoch 118: val_accuracy did not improve from 0.96149
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 504ms/step - accuracy: 0.9994 - loss: 0.0184 - val_accuracy: 0.9574 - val_loss: 0.1198 - learning_rate: 1.0000e-06
Epoch 119/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 0.9987 - loss: 0.0214
Epoch 119: val_accuracy improved from 0.96149 to 0.96286, saving model to C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Re

106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 506ms/step - accuracy: 0.9987 - loss: 0.0214 - val_accuracy: 0.9629 - val_loss: 0.1017 - learning_rate: 1.0000e-06
Epoch 120/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.9972 - loss: 0.0169
Epoch 120: val_accuracy did not improve from 0.96286
106/106 ━━━━━━━━━━━━━━━━━━━━ 82s 502ms/step - accuracy: 0.9972 - loss: 0.0169 - val_accuracy: 0.9560 - val_loss: 0.1532 - learning_rate: 1.0000e-06
Epoch 121/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 0.9981 - loss: 0.0241
Epoch 121: val_accuracy did not improve from 0.96286
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 502ms/step - accuracy: 0.9981 - loss: 0.0241 - val_accuracy: 0.9574 - val_loss: 0.1109 - learning_rate: 1.0000e-06
Epoch 122/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9980 - loss: 0.0194
Epoch 122: val_accuracy did not improve from 0.96286
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 504ms/step - accuracy: 0.9980 - loss: 0.0194 - val_accuracy: 0.9615 - val_loss: 0.1149 - le

106/106 ━━━━━━━━━━━━━━━━━━━━ 55s 508ms/step - accuracy: 0.9991 - loss: 0.0077 - val_accuracy: 0.9670 - val_loss: 0.0994 - learning_rate: 1.0000e-06
Epoch 165/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 1.0000 - loss: 0.0072
Epoch 165: val_accuracy did not improve from 0.96699
106/106 ━━━━━━━━━━━━━━━━━━━━ 82s 504ms/step - accuracy: 1.0000 - loss: 0.0072 - val_accuracy: 0.9601 - val_loss: 0.1099 - learning_rate: 1.0000e-06
Epoch 166/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 1.0000 - loss: 0.0056
Epoch 166: val_accuracy improved from 0.96699 to 0.96974, saving model to C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\models_cnn_traj\traj_cnn_3cls_20260112-041725.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 506ms/step - accuracy: 1.0000 - loss: 0.0056 - val_accuracy: 0.9697 - val_loss: 0.1003 - learning_rate: 1.0000e-06
Epoch 167/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 1.0000 - loss: 0.0102
Epoch 167: val_accuracy did not improve from 0.96974
106/106 ━━━━━━━━━━━━━━━━━━━━ 82s 502ms/step - accuracy: 1.0000 - loss: 0.0101 - val_accuracy: 0.9684 - val_loss: 0.0901 - learning_rate: 1.0000e-06
Epoch 168/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 1.0000 - loss: 0.0059
Epoch 168: val_accuracy did not improve from 0.96974
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 503ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.9601 - val_loss: 0.0982 - learning_rate: 1.0000e-06
Epoch 169/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 1.0000 - loss: 0.0052
Epoch 169: val_accuracy did not improve from 0.96974
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 503ms/step - accuracy: 0.9999 - loss: 0.0053 - val_accuracy: 0.9642 - val_loss: 0.1026 - le

106/106 ━━━━━━━━━━━━━━━━━━━━ 55s 508ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.9711 - val_loss: 0.0836 - learning_rate: 1.0000e-06
Epoch 188/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 1.0000 - loss: 0.0045
Epoch 188: val_accuracy did not improve from 0.97111
106/106 ━━━━━━━━━━━━━━━━━━━━ 82s 504ms/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 0.9629 - val_loss: 0.1052 - learning_rate: 1.0000e-06
Epoch 189/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 1.0000 - loss: 0.0049
Epoch 189: val_accuracy did not improve from 0.97111
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 504ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.9684 - val_loss: 0.0840 - learning_rate: 1.0000e-06
Epoch 190/200
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 1.0000 - loss: 0.0067
Epoch 190: val_accuracy did not improve from 0.97111
106/106 ━━━━━━━━━━━━━━━━━━━━ 54s 504ms/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 0.9601 - val_loss: 0.0997 - le